In [1]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "data.csv"

data = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "lonnieqin/englishspanish-translation-dataset",
  file_path,
)



<ipython-input-1-5d10f46fa241>:6: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  data = kagglehub.load_dataset(


100%|██████████| 2.72M/2.72M [00:00<00:00, 159MB/s]

Extracting zip of data.csv...


In [2]:
data.head()

,english,spanish
0,Go.,Ve.
1,Go.,Vete.
2,Go.,Vaya.
3,Go.,Váyase.
4,Hi.,Hola.


In [3]:
data.describe()

,english,spanish
count,118964,118964
unique,102904,113322
top,I could kill you.,Ahora tengo que irme.
freq,24,9


In [4]:
data.drop_duplicates(inplace=True)

In [5]:
data.shape

(118964, 2)

In [6]:
train_data = data[-1250:-250]
eval_data = data[-250:]


print(eval_data.shape)

(250, 2)


In [7]:
train_data.shape

(1000, 2)

In [8]:
eval_data.head()

,english,spanish
118714,It wasn't until long after they got married th...,No fue hasta mucho después de casarse que Tom ...
118715,Mary waited years for Tom to get out of prison...,Mary estuvo años esperando que Tom salga de la...
118716,Never work on electrical wiring without unplug...,Jamás trabajes con cableado eléctrico sin desc...
118717,Professional translators quite often specializ...,Los traductores profesionales muy a menudo se ...
118718,The bursting of Japan's so-called bubble econo...,El estallido de la así llamada burbuja especul...


In [9]:
train_data.to_csv('train_data.csv', index=False)
eval_data.to_csv('eval_data.csv', index=False)

In [10]:
!pip install -q torch peft bitsandbytes transformers trl accelerate einops tqdm scipy triton unsloth xformers cut_cross_entropy unsloth_zoo sentencepiece protobuf datasets huggingface_hub hf_transfer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [11]:
from datasets import load_dataset

data_files = {
    "train": "train_data.csv",
    "eval": "eval_data.csv",
}
data = load_dataset('csv', data_files=data_files, split=['train','eval'])
data


Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

[Dataset({
     features: ['english', 'spanish'],
     num_rows: 1000
 }),
 Dataset({
     features: ['english', 'spanish'],
     num_rows: 250
 })]

In [12]:
import os
import torch
from unsloth import FastLanguageModel
from trl import SFTTrainer, SFTConfig, DataCollatorForCompletionOnlyLM
from unsloth import is_bfloat16_supported
from tqdm.notebook import tqdm
from huggingface_hub import interpreter_login

interpreter_login()


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) n


In [13]:
max_seq_length = 2048
dtype=None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name='unsloth/Qwen2.5-3B-Instruct-unsloth-bnb-4bit',
    max_seq_length=max_seq_length,
    load_in_4bit=load_in_4bit,
    dtype=dtype,
    device_map="auto"

)


==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.36G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.36k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [37]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [14]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha = 32,
    target_modules = ['q_proj','v_proj','k_proj','o_proj','gate_proj','up_proj','down_proj'],
    lora_dropout=0.05,
    modules_to_save=["lm_head", "embed_tokens"],
    bias='none',
    use_gradient_checkpointing=True,

)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.3.19 patched 36 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM


In [16]:
translation_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that completes the request.
### Instruction :
Translate input text from english to spanish
### Input :
{input}
### Response :
{output}
"""
EOS_TOKEN = tokenizer.eos_token
def formatting_prompt_func(example):
    inputs = example["english"]
    outputs = example["spanish"]
    texts = []
    for input, output in zip(inputs, outputs):
        prompt = translation_prompt.format(input=input, output=output) + EOS_TOKEN
        texts.append(prompt)

    return texts


In [25]:
training_arguments = SFTConfig(
    output_dir='training',
    do_eval=True,
    eval_strategy='steps',
    eval_steps=200,
    logging_steps = 10,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=1,
    learning_rate = 5e-4,
    weight_decay = 0.01,
    num_train_epochs = 2,
    lr_scheduler_type='cosine',
    warmup_ratio=0.05,
    optim='paged_adamw_8bit',
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),

)


trainer = SFTTrainer(
    model=model,
    train_dataset=data[0],
    eval_dataset=data[1],
    formatting_func=formatting_prompt_func,
    args=training_arguments,
    processing_class = tokenizer
)

trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 2 | Total steps = 1,000
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 652,263,424/3,000,000,000 (21.74% trained)


Step,Training Loss,Validation Loss
200,1.165500,1.755142
400,1.399200,1.719875
600,0.468600,1.759520
800,0.418500,1.605003
1000,0.430800,1.602554


TrainOutput(global_step=1000, training_loss=0.9575839591026306, metrics={'train_runtime': 1486.2415, 'train_samples_per_second': 1.346, 'train_steps_per_second': 0.673, 'total_flos': 3254802768150528.0, 'train_loss': 0.9575839591026306})

In [35]:
def inference(text, model, tokenizer, max_input_tokens=2048, max_output_tokens=1000):
  # Tokenize
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
  )

  # Generate
  device = model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    max_length=max_output_tokens
  )

  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(text):]

  return generated_text_answer

text = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that completes the request.
### Instruction :
Translate input text from english to spanish
### Input :
i will deploy you on github.
### Response :

  """
resp = inference(text, model, tokenizer)
print(resp)

 Me pondré a trabajar en el proyecto de tu nombre en el sitio web de desarrollo de aplicación web.



In [38]:

import os



repo_name = "Amen002365/Qwen2.5-English-to-Spanish-finetuned"

model.push_to_hub(repo_name, use_auth_token=True)
tokenizer.push_to_hub(repo_name, use_auth_token=True)

README.md:   0%|          | 0.00/611 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/2.61G [00:00<?, ?B/s]

Saved model to https://huggingface.co/Amen002365/Qwen2.5-English-to-Spanish-finetuned


tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [ ]:
from peft import PeftModel, PeftConfig

max_seq_length = 2048
dtype=None
load_in_4bit = True
base_model, base_tokenizer = FastLanguageModel.from_pretrained(
    model_name='unsloth/Qwen2.5-3B-Instruct-unsloth-bnb-4bit',
    max_seq_length=max_seq_length,
    load_in_4bit=load_in_4bit,
    dtype=dtype,
    device_map="auto"

)


adapter_path = "/content/training/checkpoint-1000"
peft_model = PeftModel.from_pretrained(base_model, adapter_path)

merged_model = peft_model.merge_and_unload()

output_dir = "finetuned_model"
merged_model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)